In [10]:
import sqlite3
import pandas as pd

csv_path = 'simulated_data.csv'
# read the csv file from its path as pandas DataFrame
data = pd.read_csv(csv_path)

db_name = 'customerStatisfaction.db'
# establish the connection with database in sqlite3
conn = sqlite3.connect(db_name)
# create a cursor object before executing SQL commands
curs = conn.cursor()
# Save the DataFrame into an SQLite table
# if table exists, it will be replaced
data.to_sql('simulated_data_table', conn, if_exists='replace', index=False)
# close the database connection
conn.close()

print(f"Data from '{csv_path}' has been successfully saved to the database '{db_name}'!")


Data from 'simulated_data.csv' has been successfully saved to the database 'customerStatisfaction.db'!


In [12]:
# Reconnect to the SQLite database
conn = sqlite3.connect(db_name)

# Query the data
# Fetch the first 5 rows
query = "SELECT * FROM simulated_data_table LIMIT 5;"
# Execute the SQL query and load the results into a pandas DataFrame
result = pd.read_sql(query, conn)

conn.close()
print(result)

   customer_id  age  income  purchase_history  customer_satisfaction
0            1   52   79061              3467                      8
1            2   39   93548              3288                      7
2            3   44   48476              5046                      7
3            4   57   63092              4041                      8
4            5   53   67105              4919                      9


In [13]:
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

print("\nFirst Customer's Age and Satisfaction:")
c.execute("SELECT age, customer_satisfaction FROM simulated_data_table ORDER BY age")
# Fetch the first result
print(c.fetchone())

conn.close()


First Customer's Age and Satisfaction:
(4, 5)


In [14]:
# Fetch all results and iterate through them
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# Extract all results from the table
c.execute("SELECT * FROM simulated_data_table ORDER BY age LIMIT 10")
listOfResults = c.fetchall()

# Print the first 10 results by looping through the extracted items
print("\nFirst 10 results in the Table:")
for item in listOfResults:
    print(item)

conn.close()


First 10 results in the Table:
(590, 4, 79145, 6959, 5)
(686, 6, 57365, 5731, 4)
(272, 7, 120895, 4915, 5)
(428, 7, 97663, 5792, 5)
(335, 8, 44076, 5246, 4)
(486, 8, 61385, 4290, 4)
(21, 9, 99492, 3086, 4)
(548, 9, 57341, 5891, 4)
(851, 9, 61933, 5677, 4)
(529, 11, 26168, 4955, 4)


In [15]:
#Aggregate Query to find out the average customer satisfaction by Age group
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# SQL query to calculate the average customer satisfaction by age group (Rounded off to 4 d.p.)
print("\nAverage Customer Satisfaction by Age Group:")
query = """
SELECT
    CASE
        WHEN age BETWEEN 18 AND 24 THEN '18-24'
        WHEN age BETWEEN 25 AND 34 THEN '25-34'
        WHEN age BETWEEN 35 AND 44 THEN '35-44'
        WHEN age BETWEEN 45 AND 54 THEN '45-54'
        ELSE '55+'
    END AS age_group,
    ROUND(AVG(customer_satisfaction), 4) AS avg_satisfaction
FROM simulated_data_table
GROUP BY age_group
ORDER BY age_group;
"""

c.execute(query)
for row in c.fetchall():
    print(row)

conn.close()


Average Customer Satisfaction by Age Group:
('18-24', 5.5254)
('25-34', 6.3956)
('35-44', 7.209)
('45-54', 8.1271)
('55+', 6.1231)


In [16]:
# Filtering Data with high income customers
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# Query to find customers with income above a certain threshold
print("\nHigh Income Customers (Income > 50000):")
c.execute("SELECT customer_id, income FROM simulated_data_table WHERE income > 50000 ORDER BY income DESC LIMIT 5")
for row in c.fetchall():
    print(row)

conn.close()


High Income Customers (Income > 50000):
(55, 292285)
(489, 228727)
(352, 224463)
(817, 221967)
(588, 213227)


In [17]:
# Combining Filters and Aggregates by querying satisfaction by income Bracket
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# Query to calculate average satisfaction for different income brackets
print("\nAverage Customer Satisfaction by Income Bracket:")
query = """
SELECT
    CASE
        WHEN income BETWEEN 0 AND 20000 THEN '0-20K'
        WHEN income BETWEEN 20001 AND 40000 THEN '20K-40K'
        WHEN income BETWEEN 40001 AND 60000 THEN '40K-60K'
        ELSE '60K+'
    END AS income_bracket,
    ROUND(AVG(customer_satisfaction), 4) AS avg_satisfaction
FROM simulated_data_table
GROUP BY income_bracket
ORDER BY income_bracket;
"""
c.execute(query)
for row in c.fetchall():
    print(row)

# Close the connection
conn.close()


Average Customer Satisfaction by Income Bracket:
('0-20K', 6.1667)
('20K-40K', 6.0846)
('40K-60K', 6.3808)
('60K+', 7.2246)


In [18]:
# To add queried data to the table
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# Add news columns to the original table
c.execute("ALTER TABLE simulated_data_table ADD COLUMN income_bracket TEXT")
c.execute("ALTER TABLE simulated_data_table ADD COLUMN avg_satisfaction_income_bracket REAL")
c.execute("ALTER TABLE simulated_data_table ADD COLUMN avg_customer_satisfaction REAL")
c.execute("ALTER TABLE simulated_data_table ADD COLUMN is_high_income INTEGER")
print("New columns added to the table successfully.")

New columns added to the table successfully.


In [19]:
# update the new columns in the table
# update the income_bracket column based on income ranges
c.execute("""
UPDATE simulated_data_table
SET income_bracket = CASE
    WHEN income BETWEEN 0 AND 20000 THEN '0-20K'
    WHEN income BETWEEN 20001 AND 40000 THEN '20K-40K'
    WHEN income BETWEEN 40001 AND 60000 THEN '40K-60K'
    ELSE '60K+'
END;
""")
print("Income bracket column populated.")

# Update the avg_satisfaction_income_bracket column with the average satisfaction for each income bracket
c.execute("""
UPDATE simulated_data_table
SET avg_satisfaction_income_bracket = (
    SELECT ROUND(AVG(customer_satisfaction), 4)
    FROM simulated_data_table AS sub
    WHERE sub.income_bracket = simulated_data_table.income_bracket
);
""")
print("Average customer satisfaction by income bracket column populated.")

# Update the avg_customer_satisfaction column with the overall average satisfaction
c.execute("""
UPDATE simulated_data_table
SET avg_customer_satisfaction = (
    SELECT ROUND(AVG(customer_satisfaction), 4)
    FROM simulated_data_table
);
""")
print("Overall average customer satisfaction column populated.")

# Update the is_high_income column to indicate if the income is greater than 50,000
c.execute("""
UPDATE simulated_data_table
SET is_high_income = CASE
    WHEN income > 50000 THEN 1
    ELSE 0
END;
""")
print("High-income indicator column populated.")

# save the changes to databse
conn.commit()
conn.close()

Income bracket column populated.
Average customer satisfaction by income bracket column populated.
Overall average customer satisfaction column populated.
High-income indicator column populated.


In [20]:
#View the updated table
conn = sqlite3.connect('customerStatisfaction.db')
c = conn.cursor()

# Fetch and display 10 rows to verify the updated table
print("\nUpdated Table (First 10 Rows):")
for row in c.execute("SELECT customer_id, age, income, income_bracket, avg_satisfaction_income_bracket, avg_customer_satisfaction, is_high_income FROM simulated_data_table LIMIT 10"):
    print(row)

conn.close()


Updated Table (First 10 Rows):
(1, 52, 79061, '60K+', 7.2246, 6.752, 1)
(2, 39, 93548, '60K+', 7.2246, 6.752, 1)
(3, 44, 48476, '40K-60K', 6.3808, 6.752, 0)
(4, 57, 63092, '60K+', 7.2246, 6.752, 1)
(5, 53, 67105, '60K+', 7.2246, 6.752, 1)
(6, 25, 66220, '60K+', 7.2246, 6.752, 1)
(7, 44, 78463, '60K+', 7.2246, 6.752, 1)
(8, 33, 24123, '20K-40K', 6.0846, 6.752, 0)
(9, 33, 58415, '40K-60K', 6.3808, 6.752, 1)
(10, 39, 67475, '60K+', 7.2246, 6.752, 1)


In [ ]:
# Preprocess data to training and testing sets before model training
import pandas as pd
from sklearn.model_selection import train_test_split
#for data preprocessing
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Load data from SQLite database
conn = sqlite3.connect('customerStatisfaction.db')
df = pd.read_sql_query("SELECT * FROM simulated_data_table", conn)
conn.close()

# Feature engineering
# Encode the 'income_bracket' categorical column into numeric values using LabelEncoder
# machine learning models provide better results with numeric data
df['income_bracket_encoded'] = LabelEncoder().fit_transform(df['income_bracket'])

# Features X and target y
# X is the columns used to predict target variable
X = df[['age', 'income', 'purchase_history', 'income_bracket_encoded']]  # Features
# y is the predicted variable
y = df['customer_satisfaction']

# Train-test split into 20% testing set and 80% for training set
# random_state is to ensure reproducibility of the split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Standardization ensures that features have mean 0 and standard deviation 1, improving model performance
# fit_transform is used on the training data to learn the scaling parameters and apply them on test data too
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# X_train_scaled and X_test_scaled are the processed input features ready for model training and evaluation
# y_train and y_test remain as the target variables corresponding to the training and testing sets


In [ ]:
print(X_train_scaled)
print(X_test_scaled)

[[ 1.51613612 -0.63873732 -0.04765072 -0.38859446]
 [-0.51366707 -1.01423519  0.76486413 -1.64212499]
 [ 1.11017548  0.45994605 -0.13888423  0.86493606]
 ...
 [ 1.21166564 -0.78289903  0.1777497  -0.38859446]
 [ 0.39974437  0.82965012 -2.08269453  0.86493606]
 [-1.22409819  1.90637646  0.514777    0.86493606]]
[[-9.19627710e-01 -5.53315861e-01  6.16743853e-01 -3.88594463e-01]
 [-1.22409819e+00 -6.09887150e-01 -8.61238910e-01 -3.88594463e-01]
 [-1.12260803e+00 -9.23019932e-01  1.89723291e+00 -1.64212499e+00]
 [-9.19627710e-01  7.55346300e-02 -1.44250903e-01  8.64936063e-01]
 [ 9.07195165e-01 -1.16211088e+00 -1.09904172e-01 -1.64212499e+00]
 [ 1.51613612e+00  3.25665944e+00 -8.64458916e-01  8.64936063e-01]
 [ 1.96764047e-01 -3.88355883e-01  7.21930716e-01 -3.88594463e-01]
 [-1.93452931e+00 -1.08682115e+00  6.51090584e-01 -1.64212499e+00]
 [ 9.52738874e-02 -3.25723385e-01 -1.11883939e+00 -3.88594463e-01]
 [ 6.02724686e-01 -2.81298804e-02  5.52343733e-01  8.64936063e-01]
 [ 7.04214846e-01 

In [ ]:
# Decision Tree for Classification
# Customer satisfaction nends to be classified to two categories: low and high (0, 1)
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import export_text

# Return True if customer_satisfaction >= 3 and convert Boolean to integer
# customer_satisfaction >= 3 represents high satisfaction; customer_satisfaction < 3 represents low satisfaction
# Assign to new column to store the 0 or 1 which represent the customer_satisfaction categories
df['satisfaction_segment'] = (df['customer_satisfaction'] >= 3).astype(int)

# Features (X) and target (y)
X = df[['age', 'income', 'purchase_history', 'income_bracket_encoded']]  # Features
y = df['satisfaction_segment']  # Target variable for classification

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Decision Tree Classifier
dt_clf = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_clf.fit(X_train, y_train)

# Predict on test data
y_pred = dt_clf.predict(X_test)

# Evaluate the model
print("Decision Tree Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Display the decision tree structure
tree_rules = export_text(dt_clf, feature_names=list(X.columns))
print("\nDecision Tree Rules:")
print(tree_rules)


Decision Tree Results:
Accuracy: 1.0000

Classification Report:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       200

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200


Decision Tree Rules:
|--- class: 0



In [ ]:
#Additional exploration: Keras model
from keras.models import Sequential
from keras.layers import Dense

# Define a simple neural network
model = Sequential()
model.add(Dense(32, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))  # For regression tasks

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=50, batch_size=16, validation_split=0.2, verbose=1)

# Evaluate the model on test data
nn_mse = model.evaluate(X_test_scaled, y_test)[0]
print(f"Neural Network MSE: {nn_mse:.4f}")


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 0.8486 - mae: 0.8370 - val_loss: 0.1464 - val_mae: 0.3219
Epoch 2/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1266 - mae: 0.2870 - val_loss: 0.0629 - val_mae: 0.2116
Epoch 3/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0580 - mae: 0.2010 - val_loss: 0.0419 - val_mae: 0.1700
Epoch 4/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0366 - mae: 0.1491 - val_loss: 0.0323 - val_mae: 0.1446
Epoch 5/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0257 - mae: 0.1246 - val_loss: 0.0263 - val_mae: 0.1274
Epoch 6/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0213 - mae: 0.1092 - val_loss: 0.0220 - val_mae: 0.1126
Epoch 7/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0165 - mae: 0.0914 - val_loss: 0.0181 - val_mae: 0.1001
Epoch 8/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0130 - mae: 0.0823 - val_loss: 0.0156 - val_mae: 0.0914
Epoch 9/50
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0105 - mae: 0.0737 - va

Since 0.0008 is a very small number, it indicates that the differences between the predicted and actual values are minimal, meaning the Neural Network is performing well.